### 5b. Text Features

In [1]:
import pickle
import pandas as pd
import numpy as np

import time

from keras.layers import LSTM, GRU, Bidirectional, concatenate, Dropout, Dense, Input, Conv1D, MaxPooling1D, Flatten
from keras import Model, losses
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import preprocessing as pre
import model_evaluation as me

import tensorflow as tf

In [2]:
with open('X_tr_1_text.pkl', 'rb') as file_name:
    X_tr_1_text = pickle.load(file_name) 
    
with open('X_val_1_text.pkl', 'rb') as file_name:
    X_val_1_text = pickle.load(file_name)
    
with open('y_tr_1.pkl', 'rb') as file_name:
    y_tr_1 = pickle.load(file_name)
    
with open('y_val_1.pkl', 'rb') as file_name:
    y_val_1 = pickle.load(file_name)

with open('X_tr_full_text.pkl', 'rb') as file_name:
    X_tr_full_text = pickle.load(file_name)

In [3]:
with open('X_test_text.pkl', 'rb') as file_name:
    X_test_text = pickle.load(file_name)

In [4]:
model_name = []
features_considered = []
training_time = []
val_rmse = []
val_mae = []

#### Data Preparation
As was already performed in the main notebook.

In [5]:
OUTPUT_SEQ_LEN_NAME = 7
OUTPUT_SEQ_LEN_SUMMARY = 50
OUTPUT_SEQ_LEN_DESCRIPTION = 110

In [6]:
import os

path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "Documents/adams/exam/data/glove.6B.100d.txt"
)

embeddings_index = {}

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [7]:
# name
X_train_name_emb = pre.text_embedding(X_tr_full_text.name, OUTPUT_SEQ_LEN_NAME, embeddings_index)
vectorizer_name = X_train_name_emb.embedding_prep()
X_train_name_emb.get_embedding_matrix()
embedding_layer_name = X_train_name_emb.get_embedding_layer()

# summary
X_train_summary_emb = pre.text_embedding(X_tr_full_text.summary, OUTPUT_SEQ_LEN_SUMMARY, embeddings_index)
vectorizer_summary = X_train_summary_emb.embedding_prep()
X_train_summary_emb.get_embedding_matrix()
embedding_layer_summary = X_train_summary_emb.get_embedding_layer()

# description
X_train_description_emb = pre.text_embedding(X_tr_full_text.description, OUTPUT_SEQ_LEN_DESCRIPTION, embeddings_index)
vectorizer_description = X_train_description_emb.embedding_prep()
X_train_description_emb.get_embedding_matrix()
embedding_layer_description = X_train_description_emb.get_embedding_layer()

Converted 1097 words (40 misses).
Successfully created embedding layer.
Converted 4085 words (117 misses).
Successfully created embedding layer.
Converted 6341 words (331 misses).
Successfully created embedding layer.


In [8]:
# create trainable embedding layers
embedding_layer_name_trainable = X_train_name_emb.get_embedding_layer(trainable=True)
embedding_layer_summary_trainable = X_train_summary_emb.get_embedding_layer(trainable=True)
embedding_layer_description_trainable = X_train_description_emb.get_embedding_layer(trainable=True)

Successfully created embedding layer.
Successfully created embedding layer.
Successfully created embedding layer.


In [9]:
# prepare training and validation data so that it can be fed into NNs
X_tr_1_text_name = vectorizer_name(np.array([[s] for s in X_tr_1_text.name])).numpy()
X_tr_1_text_summary = vectorizer_summary(np.array([[s] for s in X_tr_1_text.summary])).numpy()
X_tr_1_text_description = vectorizer_description(np.array([[s] for s in X_tr_1_text.description])).numpy()

X_val_1_text_name = vectorizer_name(np.array([[s] for s in X_val_1_text.name])).numpy()
X_val_1_text_summary = vectorizer_summary(np.array([[s] for s in X_val_1_text.summary])).numpy()
X_val_1_text_description = vectorizer_description(np.array([[s] for s in X_val_1_text.description])).numpy()

y_val_1 = np.array(y_val_1)

#### Models

In [10]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=0.01, restore_best_weights=True)]

#### 1.1.1 GRU - All Text Features (Static Embeddings)

In [11]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name(input_name)
inter_name = GRU(100)(embed_name) 
dense_name = Dense(1, activation="relu")(inter_name)

# summary
input_summary = Input(shape=(None, ))
embed_summary = embedding_layer_summary(input_summary)
inter_summary = GRU(200)(embed_summary) 
dense_summary = Dense(1, activation="relu")(inter_summary)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description(input_description)
inter_description = GRU(200)(embed_description) 
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_summary, dense_description])
outputs = Dense(1)(concat)

model = Model(inputs=[input_name, input_summary, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mae', me.root_mean_squared_error])

In [12]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_summary, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 37s 90ms/step - loss: 13241.9785 - mae: 79.5591 - root_mean_squared_error: 113.9163 - val_loss: 9586.2354 - val_mae: 61.4915 - val_root_mean_squared_error: 97.3611
Epoch 2/30
378/378 [==============================] - 33s 86ms/step - loss: 7851.9644 - mae: 57.5942 - root_mean_squared_error: 87.8628 - val_loss: 7056.0703 - val_mae: 58.0648 - val_root_mean_squared_error: 83.5404
Epoch 3/30
378/378 [==============================] - 33s 86ms/step - loss: 7026.0312 - mae: 59.1619 - root_mean_squared_error: 83.2101 - val_loss: 6991.1606 - val_mae: 59.7279 - val_root_mean_squared_error: 83.1966
Epoch 4/30
378/378 [==============================] - 33s 86ms/step - loss: 6391.1099 - mae: 53.8360 - root_mean_squared_error: 79.1678 - val_loss: 5472.2412 - val_mae: 45.7244 - val_root_mean_squared_error: 73.4626
Epoch 5/30
378/378 [==============================] - 33s 87ms/step - loss: 5151.6860 - mae: 45.5471 - root_mean_squared_error: 71.088

In [13]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1)

In [14]:
model_name.append('GRU (Static Emb.)')
features_considered.append('Name, Summary, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 1.1.2 GRU - All Text Features (Trainable Embeddings)

In [15]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name_trainable(input_name)
inter_name = GRU(100)(embed_name) 
dense_name = Dense(1, activation="relu")(inter_name)

# summary
input_summary = Input(shape=(None, ))
embed_summary = embedding_layer_summary_trainable(input_summary)
inter_summary = GRU(200)(embed_summary) 
dense_summary = Dense(1, activation="relu")(inter_summary)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = GRU(200)(embed_description) 
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_summary, dense_description])
outputs = Dense(1)(concat)

model = Model(inputs=[input_name, input_summary, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mae', me.root_mean_squared_error])

In [16]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_summary, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 38s 94ms/step - loss: 13278.8623 - mae: 79.7194 - root_mean_squared_error: 114.1910 - val_loss: 9623.9736 - val_mae: 61.6281 - val_root_mean_squared_error: 97.5542
Epoch 2/30
378/378 [==============================] - 35s 92ms/step - loss: 7872.3096 - mae: 57.6122 - root_mean_squared_error: 87.9241 - val_loss: 7064.7642 - val_mae: 57.9617 - val_root_mean_squared_error: 83.5889
Epoch 3/30
378/378 [==============================] - 35s 92ms/step - loss: 6595.6318 - mae: 54.4958 - root_mean_squared_error: 80.4694 - val_loss: 5501.4165 - val_mae: 45.7257 - val_root_mean_squared_error: 73.6471
Epoch 4/30
378/378 [==============================] - 35s 92ms/step - loss: 5034.1406 - mae: 44.0639 - root_mean_squared_error: 70.2760 - val_loss: 4741.0894 - val_mae: 42.6598 - val_root_mean_squared_error: 68.3592
Epoch 5/30
378/378 [==============================] - 35s 92ms/step - loss: 4497.0386 - mae: 42.1239 - root_mean_squared_error: 66.379

In [17]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1)

In [18]:
model_name.append('GRU (Trainable Emb.)')
features_considered.append('Name, Summary, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 1.2.1 GRU - Name and Description (Static Embeddings)

In [33]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name(input_name)
inter_name = GRU(100)(embed_name) 
dense_name = Dense(1, activation="relu")(inter_name)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description(input_description)
inter_description = GRU(200)(embed_description) 
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_description])
outputs = Dense(1)(concat)

model = Model(inputs=[input_name, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mae', me.root_mean_squared_error])

In [34]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=False,
                    validation_data=([X_val_1_text_name, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 27s 65ms/step - loss: 11110.9932 - mae: 70.4006 - root_mean_squared_error: 103.6702 - val_loss: 7223.4160 - val_mae: 56.9847 - val_root_mean_squared_error: 84.4911
Epoch 2/30
378/378 [==============================] - 24s 63ms/step - loss: 6673.8057 - mae: 55.5686 - root_mean_squared_error: 81.0130 - val_loss: 5629.2500 - val_mae: 48.5203 - val_root_mean_squared_error: 74.5047
Epoch 3/30
378/378 [==============================] - 23s 62ms/step - loss: 5219.3750 - mae: 47.1910 - root_mean_squared_error: 71.6174 - val_loss: 4883.9712 - val_mae: 44.7657 - val_root_mean_squared_error: 69.3433
Epoch 4/30
378/378 [==============================] - 23s 62ms/step - loss: 4551.2847 - mae: 44.1129 - root_mean_squared_error: 66.8592 - val_loss: 4249.6973 - val_mae: 39.9590 - val_root_mean_squared_error: 64.6531
Epoch 5/30
378/378 [==============================] - 23s 62ms/step - loss: 3979.5232 - mae: 39.9487 - root_mean_squared_error: 62.472

In [35]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_description], y_val_1)

In [36]:
model_name.append('GRU (Static Emb.)')
features_considered.append('Name, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 1.2.2 GRU - Name and Description (Trainable Embeddings)

In [37]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name_trainable(input_name)
inter_name = GRU(100)(embed_name) 
dense_name = Dense(1, activation="relu")(inter_name)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = GRU(200)(embed_description) 
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_description])
outputs = Dense(1)(concat)

model = Model(inputs=[input_name, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mae', me.root_mean_squared_error])

In [38]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 35s 87ms/step - loss: 11099.5635 - mae: 70.4102 - root_mean_squared_error: 103.6158 - val_loss: 7198.7803 - val_mae: 56.7915 - val_root_mean_squared_error: 84.3443
Epoch 2/30
378/378 [==============================] - 32s 84ms/step - loss: 5803.7598 - mae: 48.3075 - root_mean_squared_error: 75.0454 - val_loss: 4491.6401 - val_mae: 39.5626 - val_root_mean_squared_error: 66.4333
Epoch 3/30
378/378 [==============================] - 32s 83ms/step - loss: 3869.8865 - mae: 38.0729 - root_mean_squared_error: 61.5137 - val_loss: 3817.3181 - val_mae: 38.8337 - val_root_mean_squared_error: 61.3081
Epoch 4/30
378/378 [==============================] - 31s 83ms/step - loss: 3258.9033 - mae: 35.6979 - root_mean_squared_error: 56.4453 - val_loss: 3585.2119 - val_mae: 37.1665 - val_root_mean_squared_error: 59.3703
Epoch 5/30
378/378 [==============================] - 31s 83ms/step - loss: 2949.7390 - mae: 34.2424 - root_mean_squared_error: 53.765

In [39]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_description], y_val_1)

In [40]:
model_name.append('GRU (Trainable Emb.)')
features_considered.append('Name, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 1.3.1 GRU - Description (Static Embeddings)

In [41]:
# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description(input_description)
inter_description = GRU(200)(embed_description) 
outputs = Dense(1)(inter_description)

model = Model(inputs=input_description, outputs=outputs)
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mae', me.root_mean_squared_error])

In [42]:
start = time.time()

history = model.fit(X_tr_1_text_description, y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=(X_val_1_text_description, y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 24s 60ms/step - loss: 10500.6006 - mae: 66.6787 - root_mean_squared_error: 101.2467 - val_loss: 7766.4541 - val_mae: 56.7684 - val_root_mean_squared_error: 87.5831
Epoch 2/30
378/378 [==============================] - 22s 58ms/step - loss: 7238.5552 - mae: 57.2834 - root_mean_squared_error: 84.3858 - val_loss: 7011.9570 - val_mae: 58.6024 - val_root_mean_squared_error: 83.2954
Epoch 3/30
378/378 [==============================] - 22s 58ms/step - loss: 6393.7583 - mae: 52.8744 - root_mean_squared_error: 79.2288 - val_loss: 5680.0679 - val_mae: 47.1508 - val_root_mean_squared_error: 74.8093
Epoch 4/30
378/378 [==============================] - 22s 58ms/step - loss: 5358.3745 - mae: 46.4309 - root_mean_squared_error: 72.4205 - val_loss: 4993.5977 - val_mae: 43.7733 - val_root_mean_squared_error: 70.1192
Epoch 5/30
378/378 [==============================] - 22s 58ms/step - loss: 4767.3320 - mae: 43.7404 - root_mean_squared_error: 68.375

In [43]:
rmse, mae = me.compute_metrics(model, X_val_1_text_description, y_val_1)

In [44]:
model_name.append('GRU (Static Emb.)')
features_considered.append('Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 1.3.2 GRU - Description (Trainable Embeddings)

In [45]:
# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = GRU(200)(embed_description) 
outputs = Dense(1)(inter_description)

model = Model(inputs=input_description, outputs=outputs)
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mae', me.root_mean_squared_error])

In [46]:
start = time.time()

history = model.fit(X_tr_1_text_description, y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=(X_val_1_text_description, y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 32s 80ms/step - loss: 10479.9590 - mae: 66.4706 - root_mean_squared_error: 101.0998 - val_loss: 7765.9717 - val_mae: 56.5703 - val_root_mean_squared_error: 87.5782
Epoch 2/30
378/378 [==============================] - 29s 78ms/step - loss: 7136.9229 - mae: 56.2785 - root_mean_squared_error: 83.7756 - val_loss: 6485.0723 - val_mae: 52.8555 - val_root_mean_squared_error: 80.0119
Epoch 3/30
378/378 [==============================] - 29s 78ms/step - loss: 5568.9678 - mae: 45.7268 - root_mean_squared_error: 73.8228 - val_loss: 5076.9165 - val_mae: 42.4721 - val_root_mean_squared_error: 70.6641
Epoch 4/30
378/378 [==============================] - 29s 78ms/step - loss: 4654.5444 - mae: 41.7771 - root_mean_squared_error: 67.5302 - val_loss: 4549.5371 - val_mae: 43.2260 - val_root_mean_squared_error: 66.9354
Epoch 5/30
378/378 [==============================] - 29s 78ms/step - loss: 4122.4263 - mae: 39.6346 - root_mean_squared_error: 63.556

In [47]:
rmse, mae = me.compute_metrics(model, X_val_1_text_description, y_val_1)

In [48]:
model_name.append('GRU (Trainable Emb.)')
features_considered.append('Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 2.1 LSTM - All Text Features 
* Adam optimizer was stuck for several epochs, RMSprop proved to converge much quicker and was thus chosen over Adam

In [49]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name_trainable(input_name)
inter_name = LSTM(100, return_sequences=False)(embed_name) 
dense_name = Dense(1, activation="relu")(inter_name)

# summary
input_summary = Input(shape=(None, ))
embed_summary = embedding_layer_summary_trainable(input_summary)
inter_summary = LSTM(200, return_sequences=False)(embed_summary) 
dense_summary = Dense(1, activation="relu")(inter_summary)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = LSTM(200, return_sequences=False)(embed_description) 
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_summary, dense_description])
outputs = Dense(1)(concat)

model = Model(inputs=[input_name, input_summary, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "RMSprop", metrics=['mae', me.root_mean_squared_error])

In [50]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_summary, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 47s 114ms/step - loss: 13319.8770 - mae: 79.9007 - root_mean_squared_error: 114.4706 - val_loss: 10030.9619 - val_mae: 63.1273 - val_root_mean_squared_error: 99.6140
Epoch 2/30
378/378 [==============================] - 40s 106ms/step - loss: 7583.4180 - mae: 53.6805 - root_mean_squared_error: 85.9350 - val_loss: 5712.1143 - val_mae: 46.5635 - val_root_mean_squared_error: 75.0287
Epoch 3/30
378/378 [==============================] - 40s 106ms/step - loss: 5018.3267 - mae: 44.5720 - root_mean_squared_error: 70.1237 - val_loss: 4736.6553 - val_mae: 44.8689 - val_root_mean_squared_error: 68.3940
Epoch 4/30
378/378 [==============================] - 39s 104ms/step - loss: 4380.8403 - mae: 42.3090 - root_mean_squared_error: 65.5781 - val_loss: 4471.7339 - val_mae: 43.5804 - val_root_mean_squared_error: 66.4415
Epoch 5/30
378/378 [==============================] - 39s 104ms/step - loss: 4152.0796 - mae: 41.2483 - root_mean_squared_error: 

In [51]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1)

In [52]:
model_name.append('LSTM')
features_considered.append('Name, Summary, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 2.2 LSTM - Name and Description

In [53]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name_trainable(input_name)
inter_name = LSTM(100)(embed_name) 
dense_name = Dense(1, activation="relu")(inter_name)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = LSTM(200)(embed_description) 
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_description])
outputs = Dense(1)(concat)

model = Model(inputs=[input_name, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "RMSprop", metrics=['mae', me.root_mean_squared_error])

In [54]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 37s 92ms/step - loss: 10929.4961 - mae: 68.5622 - root_mean_squared_error: 102.8287 - val_loss: 6407.2271 - val_mae: 48.5280 - val_root_mean_squared_error: 79.4736
Epoch 2/30
378/378 [==============================] - 33s 88ms/step - loss: 4843.3452 - mae: 43.0619 - root_mean_squared_error: 68.7862 - val_loss: 4217.3608 - val_mae: 39.7258 - val_root_mean_squared_error: 64.4543
Epoch 3/30
378/378 [==============================] - 34s 89ms/step - loss: 3773.5986 - mae: 38.6942 - root_mean_squared_error: 60.7737 - val_loss: 3791.4172 - val_mae: 39.4981 - val_root_mean_squared_error: 61.1667
Epoch 4/30
378/378 [==============================] - 33s 88ms/step - loss: 3369.5164 - mae: 36.5000 - root_mean_squared_error: 57.4160 - val_loss: 3547.9150 - val_mae: 37.8963 - val_root_mean_squared_error: 59.1191
Epoch 5/30
378/378 [==============================] - 34s 89ms/step - loss: 3131.4775 - mae: 35.4060 - root_mean_squared_error: 55.336

In [55]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_description], y_val_1)

In [56]:
model_name.append('LSTM')
features_considered.append('Name, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 2.3 LSTM - Description

In [57]:
# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = LSTM(200)(embed_description) 
outputs = Dense(1)(inter_description)

model = Model(inputs=input_description, outputs=outputs)
model.compile(loss = 'mean_squared_error', optimizer = "RMSprop", metrics=['mae', me.root_mean_squared_error])

In [58]:
start = time.time()

history = model.fit(X_tr_1_text_description, y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=(X_val_1_text_description, y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 35s 85ms/step - loss: 9876.2598 - mae: 63.7285 - root_mean_squared_error: 98.0827 - val_loss: 6751.7388 - val_mae: 50.8692 - val_root_mean_squared_error: 81.6110
Epoch 2/30
378/378 [==============================] - 31s 82ms/step - loss: 5566.6055 - mae: 45.8829 - root_mean_squared_error: 73.8560 - val_loss: 4732.2500 - val_mae: 42.8959 - val_root_mean_squared_error: 68.2399
Epoch 3/30
378/378 [==============================] - 31s 82ms/step - loss: 4411.5151 - mae: 41.4995 - root_mean_squared_error: 65.6060 - val_loss: 4527.3950 - val_mae: 46.5473 - val_root_mean_squared_error: 66.8547
Epoch 4/30
378/378 [==============================] - 31s 83ms/step - loss: 4002.8230 - mae: 39.9809 - root_mean_squared_error: 62.5953 - val_loss: 4095.6401 - val_mae: 42.4061 - val_root_mean_squared_error: 63.5409
Epoch 5/30
378/378 [==============================] - 31s 82ms/step - loss: 3755.9316 - mae: 38.6901 - root_mean_squared_error: 60.6420 

In [59]:
rmse, mae = me.compute_metrics(model, X_val_1_text_description, y_val_1)

In [60]:
model_name.append('LSTM')
features_considered.append('Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 3. Bidirectional LSTM - All Text Features

In [61]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name_trainable(input_name)
inter_name = Bidirectional(LSTM(100, return_sequences=False), merge_mode="concat")(embed_name) 
dense_name = Dense(1, activation="relu")(inter_name)

# summary
input_summary = Input(shape=(None, ))
embed_summary = embedding_layer_summary_trainable(input_summary)
inter_summary = Bidirectional(LSTM(200, return_sequences=False), merge_mode="concat")(embed_summary) 
dense_summary = Dense(1, activation="relu")(inter_summary)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = Bidirectional(LSTM(200, return_sequences=False), merge_mode="concat")(embed_description) 
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_summary, dense_description])
outputs = Dense(1)(concat)


model = Model(inputs=[input_name, input_summary, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "RMSprop", metrics=['mae', me.root_mean_squared_error])

In [62]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_summary, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 98s 240ms/step - loss: 10934.6748 - mae: 68.4941 - root_mean_squared_error: 103.1966 - val_loss: 6772.4722 - val_mae: 49.3531 - val_root_mean_squared_error: 81.7173
Epoch 2/30
378/378 [==============================] - 83s 220ms/step - loss: 5177.1987 - mae: 44.1416 - root_mean_squared_error: 71.1669 - val_loss: 4592.7485 - val_mae: 43.2334 - val_root_mean_squared_error: 67.3182
Epoch 3/30
378/378 [==============================] - 83s 219ms/step - loss: 4231.1846 - mae: 41.2533 - root_mean_squared_error: 64.3830 - val_loss: 4345.7656 - val_mae: 41.8307 - val_root_mean_squared_error: 65.4736
Epoch 4/30
378/378 [==============================] - 83s 219ms/step - loss: 4008.4692 - mae: 40.2473 - root_mean_squared_error: 62.6265 - val_loss: 4438.8828 - val_mae: 40.3284 - val_root_mean_squared_error: 66.1106
Epoch 5/30
378/378 [==============================] - 83s 219ms/step - loss: 3906.0750 - mae: 39.7026 - root_mean_squared_error: 6

In [63]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1)

In [64]:
model_name.append('Bidirect. LSTM')
features_considered.append('Name, Summary, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 4. Deep LSTM with Dropout Regularization - All Text Features

In [65]:
# name
input_name = Input(shape=(None, ))
embed_name = embedding_layer_name_trainable(input_name)
inter_name = LSTM(100, return_sequences=True)(embed_name) 
inter_name = Dropout(0.05)(inter_name)
inter_name = LSTM(50, return_sequences=False)(inter_name)
inter_name = Dropout(0.05)(inter_name)
dense_name = Dense(1, activation="relu")(inter_name)

# summary
input_summary = Input(shape=(None, ))
embed_summary = embedding_layer_summary_trainable(input_summary)
inter_summary = LSTM(100, return_sequences=True)(embed_summary) 
inter_summary = Dropout(0.05)(inter_summary)
inter_summary = LSTM(50, return_sequences=False)(inter_summary)
inter_summary = Dropout(0.05)(inter_summary)
dense_summary = Dense(1, activation="relu")(inter_summary)

# description
input_description = Input(shape=(None, ))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = LSTM(100, return_sequences=True)(embed_description) 
inter_description = Dropout(0.05)(inter_description)
inter_description = LSTM(50, return_sequences=False)(inter_description)
inter_description = Dropout(0.05)(inter_description)
dense_description = Dense(1, activation="relu")(inter_description)


# concatenate individual models
concat = concatenate([dense_name, dense_summary, dense_description])
outputs = Dense(1)(concat)

model = Model(inputs=[input_name, input_summary, input_description], outputs = outputs)
model.compile(loss = 'mean_squared_error', optimizer = "RMSprop", metrics=['mae', me.root_mean_squared_error])

In [66]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_summary, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128, 
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 68s 161ms/step - loss: 14911.2480 - mae: 88.7356 - root_mean_squared_error: 121.4979 - val_loss: 12789.4551 - val_mae: 76.8077 - val_root_mean_squared_error: 112.6058
Epoch 2/30
378/378 [==============================] - 54s 142ms/step - loss: 10585.1602 - mae: 66.0041 - root_mean_squared_error: 102.0884 - val_loss: 8562.4062 - val_mae: 58.2877 - val_root_mean_squared_error: 91.9759
Epoch 3/30
378/378 [==============================] - 54s 143ms/step - loss: 6985.2188 - mae: 50.8105 - root_mean_squared_error: 82.6481 - val_loss: 5684.8179 - val_mae: 46.2125 - val_root_mean_squared_error: 74.8634
Epoch 4/30
378/378 [==============================] - 55s 144ms/step - loss: 4992.1870 - mae: 42.9920 - root_mean_squared_error: 69.8670 - val_loss: 4635.3242 - val_mae: 43.1011 - val_root_mean_squared_error: 67.6208
Epoch 5/30
378/378 [==============================] - 59s 157ms/step - loss: 4333.3774 - mae: 41.2241 - root_mean_squared_erro

In [67]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_summary, X_val_1_text_description], y_val_1)

In [68]:
model_name.append('Deep LSTM')
features_considered.append('Name, Summary, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### 6. Convolutional Neural Network - Name and Description

In [69]:
# name 
input_name = Input(shape=(X_tr_1_text_name.shape[1],))
embed_name = embedding_layer_name_trainable(input_name)
inter_name = Conv1D(filters=64, kernel_size=7, activation='relu')(embed_name)
inter_name = MaxPooling1D(pool_size=1)(inter_name)
inter_name = Flatten()(inter_name)
dense_name = Dense(1, activation='relu')(inter_name)

# description
input_description = Input(shape=(X_tr_1_text_description.shape[1],))
embed_description = embedding_layer_description_trainable(input_description)
inter_description = Conv1D(filters=128, kernel_size=5, activation='relu')(embed_description)
inter_description = MaxPooling1D(pool_size=2)(inter_description)
inter_description = Conv1D(filters=64, kernel_size=5, activation='relu')(inter_description)
inter_description = MaxPooling1D(pool_size=2)(inter_description)
inter_description = Conv1D(filters=32, kernel_size=5, activation='relu')(inter_description)
inter_description = MaxPooling1D(pool_size=2)(inter_description)
inter_description = Flatten()(inter_description)
dense_description = Dense(1, activation='relu')(inter_description)

# concatenate individual models
concat = concatenate([dense_name, dense_description])
inter = Dropout(0.1)(concat)
outputs = Dense(1)(inter)

model = Model(inputs=[input_name, input_description], outputs=outputs)
model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mae', me.root_mean_squared_error])

In [70]:
start = time.time()

history = model.fit([X_tr_1_text_name, X_tr_1_text_description], y_tr_1,
                    epochs=30,
                    verbose=1, 
                    batch_size=128,
                    shuffle=True,
                    validation_data=([X_val_1_text_name, X_val_1_text_description], y_val_1),
                    callbacks=callbacks,
                    workers=8,
                    use_multiprocessing=True)

runtime = round((time.time() - start)/60, 2)

Epoch 1/30
378/378 [==============================] - 21s 50ms/step - loss: 5413.1064 - mae: 48.7984 - root_mean_squared_error: 72.2977 - val_loss: 3785.3850 - val_mae: 39.9526 - val_root_mean_squared_error: 61.0596
Epoch 2/30
378/378 [==============================] - 17s 44ms/step - loss: 4230.7651 - mae: 43.0247 - root_mean_squared_error: 64.5125 - val_loss: 3780.2114 - val_mae: 37.9718 - val_root_mean_squared_error: 60.9636
Epoch 3/30
378/378 [==============================] - 16s 43ms/step - loss: 4017.7610 - mae: 41.7095 - root_mean_squared_error: 62.7233 - val_loss: 3504.7268 - val_mae: 38.2481 - val_root_mean_squared_error: 58.7453
Epoch 4/30
378/378 [==============================] - 16s 44ms/step - loss: 3833.9185 - mae: 40.5666 - root_mean_squared_error: 61.3347 - val_loss: 3519.7544 - val_mae: 36.5753 - val_root_mean_squared_error: 58.8259
Epoch 5/30
378/378 [==============================] - 16s 43ms/step - loss: 3752.9019 - mae: 39.8555 - root_mean_squared_error: 60.6408 

In [71]:
rmse, mae = me.compute_metrics(model, [X_val_1_text_name, X_val_1_text_description], y_val_1)

In [72]:
model_name.append('CNN')
features_considered.append('Name, Description')
training_time.append(runtime)
val_rmse.append(rmse)
val_mae.append(mae)

#### Export Results Table

In [73]:
results = pd.DataFrame({'Model': model_name,
                        'Features Considered': features_considered, 
                        'Training Time (min)': training_time,
                        'Val. RMSE': val_rmse,
                        'Val. MAE': val_mae})

In [74]:
results

,Model,Features Considered,Training Time (min),Val. RMSE,Val. MAE
0,GRU (Static Emb.),"Name, Summary, Description",7.13,66.002661,41.513946
1,GRU (Trainable Emb.),"Name, Summary, Description",7.00,64.755734,40.673919
2,GRU (Static Emb.),"Name, Description",4.35,60.282293,36.802918
3,GRU (Trainable Emb.),"Name, Description",4.79,58.608953,36.502612
4,GRU (Static Emb.),Description,5.51,60.972348,38.694965
5,GRU (Trainable Emb.),Description,8.38,60.868867,37.942431
6,LSTM,"Name, Summary, Description",14.05,64.854406,41.109050
7,LSTM,"Name, Description",3.96,59.564381,37.896297
8,LSTM,Description,7.32,60.637128,38.414291
9,Bidirect. LSTM,"Name, Summary, Description",18.22,64.557382,41.417985


In [75]:
results.to_csv('results_models_text.csv')